In [ ]:
#This file defines the patient agent, doctor agent and laboratory agent,then agents will interact with each other to simulate a medical diagnosis process.

import openai  # OpenAI API is pretty expensive,try to find something to change it
import time

In [ ]:
#delete it before submitting
openai.api_key = ""

In [ ]:
#for judging if the data is valid
import json

file_path = "MedQA_organized.jsonl"

with open(file_path, "r", encoding="utf-8") as f:
    for idx, line in enumerate(f, start=1):
        line = line.strip()
        if not line:
            continue  # 跳过空行
        try:
            obj = json.loads(line)
            print(f"Line {idx} parsed successfully:", obj)
        except json.JSONDecodeError as e:
            print(f"Line {idx} JSONDecodeError: {e}")

In [ ]:
import json

file_path = 'MedQA_organized.jsonl'
data = []
with open(file_path, 'r', encoding='utf-8') as f:
    for line in f:
        # 去掉换行符
        line = line.strip()
        # 如果这行内容不是空的，就解析成 Python 对
        data.append(json.loads(line))
        print(data)


In [ ]:
print(data[2])

In [47]:
class DataDistributor:
    """
    Structure data from a json object.
    """
    def __init__(self, json_object):
        self.patient_information = json_object['Patient Information']
        self.disease = json_object['Correct_Diagnosis']
        self.physical_examination_findings=json_object['Physical_Examination_Findings']
        self.test_result = json_object['Test_Results']
    
    def get_summary(self):
        """
        Return a summary of the structured data.
        """
        return {
            "Patient Information": self.patient_information,
            "Correct Diagnosis": self.disease,
            "Physical Examination Findings": self.physical_examination_findings,
            "Test Results": self.test_result,
        }
    

In [99]:
a=DataDistributor(data[2])


In [98]:
a.patient_informationLabor

AttributeError: 'DataDistributor' object has no attribute 'patient_informationLabor'

In [95]:
#逻辑：三个agent，病人、医生和实验室
#实验室是个函数，接受医生的测试请求，返回测试结果。如果没有相关的测试则返回正常。


class Laboratory:
    """
    Return measurement results based on the doctor's request.
    """
    prompt = "你将扮演一个用于医学检查的实验室。"
   
    def __init__(self, data_distributor):
        self.Test_Results = data_distributor.test_result
    def get_result(self, doctor_request):
        """
        Return the result of the test.
        """
        messages = [
        {"role": "system", "content": Laboratory.prompt},
        {"role": "user", "content": f"医生请求检查:{doctor_request}"+
                         f"这是检查结果的列表{self.Test_Results}"+
                         "请直接回复对应的检查结果，不需要添加额外信息"},
            ]
        response = openai.ChatCompletion.create(
                model="gpt-4o",
                messages=messages,
            )
            
        return response.choices[0].message['content']
        

In [96]:
def asking_question(model_name,prompt,system_prompt,max_retries=2,time_out=10):
    """
    ask the model and return the answer
    """
    valid_model = ["gpt4o"]
    if model_name not in valid_model:
        raise Exception(f"No model by the name {model_name},all the valid models are {valid_model}")
    
    for _ in range(max_retries):
        try:
            if model_name == "gpt4o":
                response = openai.ChatCompletion.create(
                    model="gpt-4o",
                    messages=[
                        {"role": "system", "content": system_prompt},
                        {"role": "user", "content": prompt},
                    ],
                )
                return  response["choices"][0]["message"]["content"]
            
            
        except Exception as e:
            print(e)
            time.sleep(time_out)
            continue

In [ ]:
class Patient:
    """
    Return measurement results based on the doctor's request.
    """

    
    def __init__(self, data_distributor, backend="gpt4o"):
        self.info = data_distributor.patient_information
        self.backend = backend
        self.conversation_history = "" #history of conversation
    
    def return_question(self, doctor_question)->str:
        #可以考虑增加历史对话的压缩机制
        """
        Return the answer of doctor's question.
        """
        input_prompt =  "\n下面是你的对话历史: " + self.conversation_history + "\n 下面是医生的回复: "+ doctor_question +"\n"+ "现在请继续你的对话\nPatient: "
        answer = asking_question(self.backend,input_prompt, self.system_prompt())
        self.conversation_history += doctor_question+ "\n\n" + answer + "\n\n"  #继续对话
        return answer
    
    
    def system_prompt(self) -> str:
        #we can add more templates and put them into another file
        core_prompt="""You are role-playing as a patient in a clinic. A doctor is inspecting you to identify your disease by asking questions and performing examinations. Your task is to respond naturally and concisely to the doctor's questions.
            Instructions:
            1. Strictly answer the questions based on the "Patient Information." If the answer to the doctor's question is found in the "Patient Information," respond according to its content. If it is not found in the "Patient Information," respond the doctor with normal information."
            2. Only answer in dialogue form, as if you were speaking directly to the doctor.
            2. Your responses should be brief, realistic, and limited below 4 sentences.
            3. Provide information that a patient might reasonably say in this scenario.
            4. Do not elaborate or provide excessive details unless explicitly asked."""
        
        patient_info_prompt = f"\n\nBelow is all of your information. {self.info}."
        return core_prompt + patient_info_prompt
    
    def reset(self) -> None:
        self.conversation_history = ""

#Now we can test the patient agent
a.patient_information
patient_agent = Patient(a)
patient_agent.return_question("你好，你过去有什么医疗记录吗")

gpt4o


In [101]:
#Now we can test the patient agent
a.patient_information
patient_agent = Patient(a)

In [91]:
patient_agent.return_question("你好，你过去有什么医疗记录吗")

gpt4o


'我已经有8年的2型糖尿病病史。'

In [92]:
patient_agent.conversation_history

'你好，我是你的医生\n\n您好，医生。最近我在做运动时，只要把右脚抬到胸部以上，就会感到脚底刺痛。而且用跑步机的坡度功能时，右小腿会抽筋。休息几分钟后，这些疼痛就会消失。\n\n你好，你还有什么症状吗\n\n没有其他症状了。休息后疼痛会消失，没有背痛，肌肉力量也正常。\n\n你好，你过去有什么医疗记录吗\n\n我已经有8年的2型糖尿病病史。\n\n'

In [ ]:
#some features can be added: 1.choose the backbone LLM 2.choose the prompt template 3.maximum conversation turns
#4. memory storage(maybe can be shared by the patient agent),could add summary mechanism to reduce the memory size
#jupyter notebook的问题：1.断点在最左边 2.所有待调试代码必须放在同一单元格里，否则跳转时会出错。 3.单步调试我的代码按钮不要碰。

In [108]:
class Doctor:
    """
    Doctor类用于模拟医生与病人之间的对话、提问和诊断流程。其功能包括：
    1. 根据病人回复（patient_answer）生成后续问题或检查请求；
    2. 记录并管理对话历史；(self.conversation_history)
    3. 在达到设定的最大提问次数后，结束对话并进行诊断。

    Attributes:
        num_conversation (int): 已向病人提出的问题数量。
        max_conversation (int): 医生可提出的最大问题数量，在此之后需结束对话。
        backend (str): 用于生成对话问题的语言模型名称(default:GPT4o)。
        conversation_history (str): 医患对话的历史记录。

    Methods:
        __init__(backend="gpt4o", max_converstion=20):
            初始化Doctor实例，并设置对话后端与最大提问次数。

        return_question(patient_answer: str) -> str:
            根据病人的回答生成新的提问或检查请求，若已超出最大提问次数则返回“diagnosis ended”。

        system_prompt() -> str:
            生成对患者端的系统提示语，用于指导患者如何回答医生的问题。

        reset() -> None:
            重置对话记录和问题计数器，以便进行新的诊断会话。

    Usage Example:
        >>> # 创建一个Doctor对象
        >>> doctor = Doctor(backend="gpt4o", max_conversation=5)
        >>> # 医生向病人提问
        >>> question = doctor.return_question("我最近胸口有点闷，偶尔伴随咳嗽。")
        >>> print(question)
        # 输出一段新的问题或检查请求，例如“你咳嗽有痰吗？持续了多久？”
    """
    def __init__(self, backend="gpt4o",max_conversation=10):
        """
        Doctor doesn't need initial data in chinese clinical environment
        :param max_conversation: the max conversation between patient and doctor
        """
        self.num_conversation=0
        self.max_conversation=max_conversation
        self.backend = backend
        self.conversation_history = "" #history of conversation
    

    def return_question(self, patient_answer)->str:
        #可以考虑增加历史对话的压缩机制
        """
        ask a question to the patient agent
        """
        if self.num_conversation>self.max_conversation:
            return "diagnosis ended"
        
        input_prompt =(
            f"在做出诊断前你最多只能提出 {self.max_conversation} 个问题，到目前为止，你已提出了{self.num_conversation}个问题"
            f"\n下面是你的对话历史:{self.conversation_history } "
            "\n 下面是病人本次的回复:{patient_answer} "
            "\n现在请继续你的对话\nDoctor: "
        )
        
        question = asking_question(self.backend,input_prompt, self.system_prompt())
        
        self.conversation_history += patient_answer+ "\n" + question + "\n"  #继续对话
        
        self.num_conversation+=1
        return question
    def system_prompt(self) -> str:
        #we can add more templates and put them into another file
        core_prompt="""你是一名专业的临床医生，正与一名病人进行对话。你可以通过提问来获取病人的病史、症状及其他关键信息，也可以向实验室（Laboratory）提出检查请求。实验室会根据你的请求返回对应的检查结果；若该检查结果正常且不关键，则返回“结果正常”。
        你的目标是：
        1. 通过与病人的有限次数对话（例如最多 10 次）收集足够的病情信息；
        2. 必要时向实验室发起检查请求，严格遵循下面的格式: 开具检查:[检查名称];
        3. 你也可以向助手申请身体检查，严格遵循下面的格式: 开具检查:[检查名称];
        4. 在充分了解病情后，使用  诊断结果: [具体诊断内容]给出最终诊断。
        5. 在尽可能搜集必要信息的前提下，尽可能降低总提问次数
        
        请注意以下要求：
        - 避免重复已提问的问题,如果你认为无需提出新的问题，则尽快完成诊断
        - 你的每次对话应小于3句，并保持专业、简洁；
        - 仅在需要时请求实验室检查；
        - 如果病人问到与诊断无关或暂时无关的问题，可以简短地回避或重申重点；
        - 一旦你认为信息已足够，请尽快完成诊断，避免过度询问；
        请基于以上规则展开你的问诊流程，并根据病人的回答实时调整策略。"""

        return core_prompt 
    
    def reset(self) -> None:
        self.conversation_history = ""
        self.num_conversation=0

#Now we can test the doctor agent

In [123]:
#unit Testing
doctor=Doctor(max_conversation=5)


In [124]:
doctor.max_conversation

5

In [110]:
doctor.return_question("")

gpt4o


'请告诉我您的主要症状是什么，以及这些症状持续了多长时间？'

In [111]:
doctor.return_question("我最近不太舒服，老是流鼻涕，发烧，打喷嚏")

gpt4o


'请描述一下您是否有其他相关症状，例如发烧、咳嗽、腹泻或皮疹等？'

In [112]:
doctor.return_question("我没有腹泻和皮疹，只是发烧且不停地打喷嚏")

gpt4o


'您提到有发烧、流鼻涕和打喷嚏，请问您是否有喉咙痛或咳嗽？这些症状出现前是否接触过感冒的人？'

In [113]:
doctor.return_question("我不知道是否接触过感冒的人，我的喉咙有点痛，也有点咳嗽")

gpt4o


'请告诉我这些症状持续了多长时间？在这之前是否有旅行史或季节性过敏？'

In [114]:
doctor.return_question("持续三天了，没有旅行史或者过敏")

gpt4o


'请告诉我这些症状已经持续了多长时间了？以及之前的任何旅行史或季节性过敏是否存在？'

In [115]:
doctor.return_question("持续三天了，没有旅行史或者过敏")

gpt4o


'是否在过去几天内曾经接触过有类似症状的人？此外，您有服用过什么药物或进行自我治疗吗？'

In [116]:
doctor.return_question("没有自我治疗，不知道是否接触了类似症状的人")

gpt4o


'请告诉我，您在三天内的体温最高达到多少度？另外，您在这段时间里是否有任何其他感到不适的地方，比如疲倦、肌肉酸痛或头痛？'

In [118]:
#seems over the max conversation limit
doctor.num_conversation

7

In [119]:
doctor.return_question("大约39度，疲倦，肌肉酸痛，头痛")

gpt4o


'请问您在三天内的体温最高达到多少度？另外，您在这段时间里是否有感到疲倦、肌肉酸痛或头痛之类的不适？'

In [120]:
doctor.return_question("大约39度，疲倦，肌肉酸痛，头痛")

gpt4o


'您描述的症状，包括高烧、流鼻涕、打喷嚏、喉咙痛、咳嗽、疲倦、肌肉酸痛和头痛，通常与感冒或流感类似。为了更准确的诊断，我想确认一下，您有没有任何呼吸困难或胸痛的症状？'

In [121]:
doctor.return_question("没有呼吸困难或胸痛")

gpt4o


'有没有任何呼吸困难或胸痛的症状？这将帮助我更好地了解您的情况。如果没有，请让我知道。'

In [122]:
doctor.return_question("没有呼吸困难或胸痛")

gpt4o


'对话历史已超过十轮，现在我将进行诊断。基于你描述的症状，包括流鼻涕、发烧、打喷嚏、喉咙痛、咳嗽、疲倦、肌肉酸痛和头痛，我认为你可能患有流感或上呼吸道感染。建议多休息、补充水分，并可服用退烧药和感冒药以减轻症状。若症状持续恶化或出现新的症状，请及时就医。\n\n诊断结果: 流感或上呼吸道感染。'

In [ ]:
#perfect，but there are some bug in the conversation。1.the doctor may ask similar question
#2.the doctor would like to reach the conversation limit even it is not necessary

In [129]:
conversation_history=""
total_inferences=5
#one of the problem is that the doctor agent trends
for _case in data:
    #initial data
    case=DataDistributor(_case) 
    patient_agent=Patient(case)
    laboratory=Laboratory(case)
    doctor=Doctor(max_conversation=total_inferences)
    
    #start inference
    doctor_question="Hello, I’m doctor. Could you tell me what’s been bothering you recently or if you have any symptoms you’d like to share?"    #let's hypnosis that the doctor will say the first sentence
    for round_idx in range(total_inferences+1):
    #     if round_idx == total_inferences - 1:
    #         conversation_history += "This is the final question. Please provide a diagnosis.\n  maybe we don't need
        patient_answer=patient_agent.return_question(doctor_question) #doctor ask question
        print(patient_answer)
        doctor_question=doctor.return_question(patient_answer) #patient return answer
        print(doctor_question)
    

gpt4o
Hello, doctor. Recently, I've been experiencing swelling and pain in my right knee. I also have some back pain that’s been bothering me.
gpt4o
请告诉我您此次就诊的主要不适症状以及持续了多长时间。
gpt4o
主要不适症状是右膝肿胀和疼痛，已经持续了一段时间。
gpt4o
请问您的膝盖和背部疼痛具体持续了多长时间？此外，您最近是否有过伤？
gpt4o
我的右膝疼痛和肿胀已经好几周了，背部疼痛则持续了更长的时间。我最近没有受过伤。
gpt4o
请问您膝盖和背部的疼痛具体持续了多长时间？另外，最近有没有曾经受过伤？
gpt4o
我的右膝疼痛和肿胀已经有好几周了，至于背部，疼痛时间要更长一些。我最近没有受过伤。
gpt4o
请问您膝盖疼痛是否伴有任何活动受限或无法承重的情况？此外，背痛是否在某些姿势或活动中加重？
gpt4o
是的，我的膝盖疼痛使我走路有些困难，有时候感觉无法承重。至于背痛，在长时间站立或坐着时会加重。
gpt4o
请问您膝盖疼痛是否伴有任何活动受限或无法承重的情况？此外，背痛是否在某些姿势或活动中加重？
gpt4o
是的，我的膝盖疼痛让走路变得困难，有时感觉难以承受重量。我的背痛在长时间站立或坐着时会加重。
gpt4o
我需要进行更多检查以帮助诊断。为此，我将开具一些实验室检查请求。开具检查: [膝关节X光]和[血液检查 - 血沉、C反应蛋白]。
gpt4o
I've been really concerned about my physical appearance. I'm struggling with acne and unwanted facial hair. These issues have been bothering me since I was a teenager.
gpt4o
请描述一下您的主要症状和持续时间？是否有任何明显的诱因或缓解因素？
gpt4o
关于我的主要症状，首先是从青春期开始持续的痤疮，其次是不需要的面部毛发。没有明显的诱因就是这些症状始终如一，并没有终止或改善。
gpt4o
您提到的主要症状有痤疮和不想要的面部毛发。请问您是否有月经不规律、体重增加或情绪波动等其